In [1]:
# new bro
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import csv
import pymysql
import pandas as pd


html = []
artis_html = []
# post_html =[]


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}

reg_url = str("https://www.chordindonesia.com/post-sitemap12.xml")
req = Request(url=reg_url, headers=headers) 
html = urlopen(req).read()

soup = BeautifulSoup(html, "lxml")

# print(soup.prettify()[1:100000])

page = soup.find('body')
pages = page.find_all('loc')
print(len(pages))

j = 0
i = 0

mydb = pymysql.connect(
  host="localhost",
  user="root",
  password="",
  database="sern2389_chord-gitar"
)
cur_mydb = mydb.cursor()

for x in pages:
    print(i)
    i+=1
    print(j)
#     print(x.text)
    linkkk = x.text
    slugget = linkkk.strip().split('/')[3]
    slugget = slugget.replace("chord-", "")
    print(slugget)
    
    check = None
    if check is None:
        print('none')
    query = "SELECT * FROM wp_posts WHERE post_name =%s"
    cur_mydb.execute(query, slugget)
    check = cur_mydb.fetchone()
    try:
        print(check[0])
        print("dah ada")

    except:
        j+=1
        if j >= 0:
#             if j == 500:
#                 break
            req_lagu = Request(url=x.text, headers=headers) 
            try:
                print("1")
                post_html = urlopen(req_lagu).read()
                soup = BeautifulSoup(post_html, "lxml")
                
                produk = soup.find_all("article","post")
                print("2")
                for p in produk:
                    try:
                        artis1 = p.select_one(".entry-title").get_text()
                    except:
                        artis1 = "zonk"
                    try:    
                        judul1 = p.select_one(".entry-title").get_text()
                    except:
                        judul1 = "zonk"
                    try:    
                        chord1 = p.select_one('.entry-content').get_text()
                    except:
                        chord1 = "zonk"
                    
                    print("2")
                    
                    artis = artis1.strip().split(' - ')[0]
                    judul = judul1.strip().split(' - ')[1]
                    chord = chord1.replace("Â","").replace("Â","").strip().split('Lihat Semua :')[0]

                    judul = judul.replace(artis + " ", "")

                    artis = artis.replace("Kunci Gitar ", "")

                    print(artis)
                    print(judul)
        #                 print(chord)

                    query = "SELECT * FROM artis WHERE artis=%s"
                    cur_mydb.execute(query, artis)
                    row_count = cur_mydb.rowcount

                    query = "SELECT * FROM artis WHERE artis=%s"
                    cur_mydb.execute(query, judul)
                    row_count2 = cur_mydb.rowcount

                    print('masuk')

                    if row_count != 0:
                        print('masuk1')
                        query = "SELECT id FROM artis WHERE artis=%s"
                        cur_mydb.execute(query, artis)
                        artisid = cur_mydb.fetchone()

                        cek_judul = (artisid[0], judul)
                        query = "SELECT * FROM lagu WHERE artis_id=%s && judul=%s"
                        cur_mydb.execute(query, cek_judul)
                        judul_check = cur_mydb.rowcount

                        if judul_check == 0:
                            lagu =(artisid[0], judul, chord)
                            query = "INSERT INTO lagu (artis_id, judul, chord) VALUES (%s, %s, %s)"
                            cur_mydb.execute(query, lagu)
                            mydb.commit()
                            print("tes")
                        else:
                            print("apaya")

                    elif row_count2 != 0:
                        print('masuk2')
                        query = "SELECT id FROM artis WHERE artis=%s"
                        cur_mydb.execute(query, judul)
                        artisid1 = cur_mydb.fetchone()

                        cek_judul1 = (artisid1[0], artis)
                        query = "SELECT * FROM lagu WHERE artis_id=%s && judul=%s"
                        cur_mydb.execute(query, cek_judul1)
                        judul_check1 = cur_mydb.rowcount

                        if judul_check1 == 0:
                            lagu =(artisid1[0], artis, chord)
                            query = "INSERT INTO lagu (artis_id, judul, chord) VALUES (%s, %s, %s)"
                            cur_mydb.execute(query, lagu)
                            mydb.commit()
                            print("tes")
                        else:
                            print("apaya")

                    else :
                        print('masuk3')
                        judul_lagu = judul1.replace("Kunci Gitar ", "")
                        print(judul_lagu)
                        lagu2 =(877, judul_lagu, chord)

                        query = "SELECT * FROM lagu WHERE judul=%s"
                        cur_mydb.execute(query, judul_lagu)
                        judul_check2 = cur_mydb.rowcount

                        if judul_check2 == 0:
                            query = "INSERT INTO lagu (artis_id, judul, chord) VALUES (%s, %s, %s)"
                            cur_mydb.execute(query, lagu2)
                            mydb.commit()
                        else:
                            print("apaya")
                
            except:
                print("link lagu error")
            
#     if j == 20:
#             break

555
0
0
pusakata-di-seberang-sana
none
8879
dah ada
1
0
birboy-ekas-sijuta-luka
none
1
2
2
Birboy (Ekas)
Sijuta Luka
masuk
masuk3
Birboy (Ekas) - Sijuta Luka
2
1
alma-margana-sayang-ost-dari-jendela-smp
none
1
2
2
Alma Margana
Sayang (OST. Dari Jendela SMP)
masuk
masuk1
tes
3
2
yura-yunita-tenang
none
12389
dah ada
4
2
datuk-nora-ariffin-naluri
none
8187
dah ada
5
2
esa-risty-janur-kuning
none
1
2
2
Esa Risty
Janur Kuning
masuk
masuk1
tes
6
3
syahiba-saufa-ojo-kakean-sambat
none
11031
dah ada
7
3
hendra-kumbara-sak-isone
none
4222
dah ada
8
3
anggi-candra-salah-menanam-rasa
none
16466
dah ada
9
3
budi-doremi-mesin-waktu-ost-aku-dan-mesin-waktu
none
16469
dah ada
10
3
asbak-band-hanya-ingin-bahagia
none
16468
dah ada
11
3
ungu-lesti-bismillah-cinta
none
6694
dah ada
12
3
atta-halilintar-aurel-hermansyah-hari-bahhagia
none
1
2
2
ATTA Halilintar & AUREL Hermansyah
HARI BAHHAGIA
masuk
masuk3
ATTA Halilintar & AUREL Hermansyah - HARI BAHHAGIA
13
4
maulana-ardiansyah-ibu
none
7189
dah ada
14

2
2
Adi Wibowo Popska
Nyamankan Hatiku
masuk
masuk3
Adi Wibowo Popska - Nyamankan Hatiku
110
34
justin-liee-tresno-kehalang-bondo
none
5519
dah ada
111
34
yoga-vhein-tak-bisa-marah
none
16514
dah ada
112
34
sabyan-maha-kasih
none
8095
dah ada
113
34
amstr-ada-satu-nona
none
892
dah ada
114
34
luka-aku-vs-wong-tuamu
none
6852
dah ada
115
34
fadly-rasul-feat-disa-blandina-basaba-mananti
none
2518
dah ada
116
34
yollanda-cintaku-dimenara-kembar
none
12320
dah ada
117
34
nella-nofrita-ramadhan
none
7914
dah ada
118
34
yollanda-feat-imam-usah-merayu
none
4754
dah ada
119
34
finda-fayolla-feat-ader-negro-sewu-alasan
none
1
2
2
Sewu Alasan
Finda Fayolla feat. Ader Negro
masuk
masuk3
Sewu Alasan - Finda Fayolla feat. Ader Negro
120
35
safira-inema-jutaan-sayang
none
1
2
2
Safira Inema
Jutaan Sayang
masuk
masuk1
tes
121
36
bebiraira-niat-puasa-ramadhan-dan-artinya
none
16523
dah ada
122
36
didik-budi-feat-cindi-cintya-dewi-kepastian
none
1
2
2
Didik Budi
Kepastian feat. Cindi Cintya Dewi
masuk


2
2
Ashe & FINNEAS
Till Forever Falls Apart
masuk
masuk3
Ashe & FINNEAS - Till Forever Falls Apart
211
76
justin-bieber-hold-on
none
5512
dah ada
212
76
justin-bieber-off-my-face
none
1
2
2
Justin Bieber
Off My Face
masuk
masuk1
tes
213
77
colapesce-dimartino-musica-leggerissima
none
1
2
2
Colapesce, Dimartino
Musica leggerissima
masuk
masuk3
Colapesce, Dimartino - Musica leggerissima
214
78
pendhoza-elingo-lagu-religi
none
1
2
2
Pendhoza
Elingo (Lagu Religi)
masuk
masuk1
tes
215
79
febian-puja-saiba-hujan-rindu
none
3406
dah ada
216
79
jubin-nautiyal-lut-gaye
none
1
2
2
Jubin Nautiyal
Lut Gaye
masuk
masuk3
Jubin Nautiyal - Lut Gaye
217
80
nasta-rayo-di-rantau
none
1
2
2
Nasta
Rayo Di Rantau
masuk
masuk3
Nasta - Rayo Di Rantau
218
81
ibu-kita-kartini
none
1
2
2
link lagu error
219
82
twenty-one-pilots-shy-away
none
1
2
2
Twenty One Pilots
Shy Away
masuk
masuk1
tes
220
83
jroa-byahe
none
1
2
2
JRoa
Byahe
masuk
masuk3
JRoa - Byahe
221
84
charity-gayle-thank-you-jesus-for-the-blood
none
1

2
2
Amyla
Rayo Talampau
masuk
masuk1
tes
321
119
bergek-padum-boh-syarat
none
16561
dah ada
322
119
luluk-darara-bertahan
none
6877
dah ada
323
119
for-revenge-x-stereo-wall-jakarta-hari-ini
none
3618
dah ada
324
119
twenty-one-pilots-choker
none
1
2
2
Twenty One Pilots
Choker
masuk
masuk1
tes
325
120
mahen-seamin-tak-seiman
none
6967
dah ada
326
120
ipank-dicelah-awan
none
4677
dah ada
327
120
hendra-kumbara-dalan-lurus
none
4221
dah ada
328
120
luki-safara-dingin-keringetan
none
6859
dah ada
329
120
enau-keringatku
none
3013
dah ada
330
120
enau-ijazah
none
3010
dah ada
331
120
enau-keranjang
none
3012
dah ada
332
120
enau-biang-masalah
none
3009
dah ada
333
120
teungku-hafidh-satu-cinta-sejuta-rasa
none
11225
dah ada
334
120
putih-abu-abu-nestapa
none
8890
dah ada
335
120
intan-pulau-saribu-mamintak-nyao
none
4786
dah ada
336
120
fany-zee-feat-aprilian-akhiri-luko-dihati
none
3325
dah ada
337
120
andra-respati-ini-semua-salahku
none
951
dah ada
338
120
nabila-maharani-feat-tri-suaka

2
2
Mudik Terhalang Bencana
Andra Respati
masuk
masuk2
tes
436
149
vicky-koga-cinta-dalam-doa
none
16642
dah ada
437
149
melukis-pelangi-andis-sagara
none
1
2
2
Melukis Pelangi
Andis Sagara
masuk
masuk2
tes
438
150
luki-safara-calon-mantu
none
16647
dah ada
439
150
aris-chaniago-feat-yona-irma-dendang-pusako-lamo
none
1
2
2
Dendang Pusako Lamo
Aris Chaniago feat. Yona Irma
masuk
masuk3
Dendang Pusako Lamo - Aris Chaniago feat. Yona Irma
440
151
syahiba-saufa-gethuk
none
1
2
2
Syahiba Saufa
Gethuk
masuk
masuk1
tes
441
152
rizky-febian-hingga-tua-bersama
none
16652
dah ada
442
152
frans-dek-bansaik-mangko-tasisiah
none
16655
dah ada
443
152
takbir-atta-halilintar-feat-aurel-hermansyah
none
1
2
2
Takbir
ATTA Halilintar feat. AUREL Hermansyah
masuk
masuk3
Takbir - ATTA Halilintar feat. AUREL Hermansyah
444
153
faisal-asahan-aku-diam-bukan-tak-tau
none
16677
dah ada
445
153
fauzana-feat-jaisky-rintang-samaso-bajanji
none
16678
dah ada
446
153
vicky-koga-harapan-palsu
none
16669
dah ada
447


2
2
Langit Sore
Dalam Iman Kita Berpisah
masuk
masuk1
tes


In [ ]:
print(len(post_html))

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

soup = BeautifulSoup(post_html[0], "lxml")

z = len(post_html)
print(z)

i = 0

while (i<z):
    soup.append(BeautifulSoup(post_html[i], "lxml"))
    print(i)
    i += 1

artis = []
judul = []
chord = []
i=1
produk = soup.find_all("div","post")
print(len(produk))
for p in produk:
    print(i)
    i += 1
    try:    
        artis.append(p.select_one(".post-title.entry-title").get_text())
    except:
        artis.append("zonk")
    try:    
        judul.append(p.select_one(".post-title.entry-title").get_text())
    except:
        judul.append("zonk")
    try:    
        chord.append(p.select_one('.post-body.entry-content').get_text())
    except:
        chord.append("zonk")
    
    
produk_dict = {'artiss':artis,'juduls':judul,'chords':chord}
df = pd.DataFrame(produk_dict)

df.sort_values('juduls',ascending=True)

df.to_csv("chordindo-1.csv", sep=',')
# import pandas as pd

# artis = []
# judul = []
# chord = []
# i=1
# produk = soup.find_all("article","post")

# for p in produk:
#     print(i)
#     i += 1
#     artis.append(p.select_one(".cat-links a").get_text())
#     judul.append(p.select_one(".entry-header h2").get_text())
#     try:    
#         chord.append(p.find('pre').get_text())
#     except:
#         chord.append("zonk")
    

# produk_dict = {'artiss':artis,'juduls':judul,'chords':chord}
# df = pd.DataFrame(produk_dict)

# df.sort_values('juduls',ascending=True)